### Deep GLVQ

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader,random_split

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np

from mnist_helpers import process_mnist_dataset

import copy

from sklvq import GLVQ

from DeepGLVQ import DeepGLVQ

In [3]:
import copy

##### Loading MNIST Dataset

In [8]:
batch_size = 256
validation_split = 0.2

train_data,validation_data,test_data = process_mnist_dataset(batch_size,validation_split)

train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(dataset=validation_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

##### GLVQ Loss Function

In [9]:
#This will be the dimension of the latent space
n_dims_for_classif = 10

lr = 1e-4
batch_size = 256
n_epochs = 100

n_prototypes_per_class = 16

#How large of a margin we want between classes. For larger margins use smaller gamma. 
gamma = 0

deep_glvq = DeepGLVQ(
    n_dims_for_classif = n_dims_for_classif,
    train_data = train_data,
    validation_data = validation_data,
    test_data = test_data,
    lr = lr,
    batch_size = batch_size,
    n_epochs = n_epochs,
    gamma = gamma,
    n_prototypes_per_class = n_prototypes_per_class
)

In [10]:
deep_glvq.train_encoder()

Epoch 1 / 100: 188it [00:09, 18.87it/s]


Epoch 1 validation loss: 0.37137678265571594
Epoch 1 validation accuracy: 0.9235833287239075

Epoch 1 train loss: 0.37137678265571594
Epoch 1 train accuracy: 0.9289166927337646



Epoch 2 / 100: 188it [00:10, 18.46it/s]


Epoch 2 validation loss: 0.34827208518981934
Epoch 2 validation accuracy: 0.9423333406448364

Epoch 2 train loss: 0.34827208518981934
Epoch 2 train accuracy: 0.9470000267028809



Epoch 3 / 100: 188it [00:11, 16.76it/s]


...

Epoch 98 validation loss: 0.28059643507003784
Epoch 98 validation accuracy: 0.9784166812896729

Epoch 98 train loss: 0.28059643507003784
Epoch 98 train accuracy: 0.9960625171661377



Epoch 99 / 100: 188it [00:14, 12.69it/s]


Epoch 99 validation loss: 0.28037914633750916
Epoch 99 validation accuracy: 0.9774166941642761

Epoch 99 train loss: 0.28037914633750916
Epoch 99 train accuracy: 0.9961458444595337



Epoch 100 / 100: 188it [00:14, 13.42it/s]


Epoch 100 validation loss: 0.2802566885948181
Epoch 100 validation accuracy: 0.9765833616256714

Epoch 100 train loss: 0.2802566885948181
Epoch 100 train accuracy: 0.9957291483879089

Test loss is 0.2802566885948181 and test accuracy is 0.9768999814987183
